In [1]:
import os
import sys
import time
import argparse
import geopandas as gpd
import fiona
import rasterio as rio
from rasterstats import zonal_stats
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mgwr.gwr import MGWR
from mgwr.sel_bw import Sel_BW
from mgwr.utils import shift_colormap, truncate_colormap
from datetime import datetime
import glob
import pathlib


In [2]:
folder_root = os.getcwd()
print(folder_root);

e:\citarum\GeoAnalisisGEE


In [3]:
desa_polygon_file = os.path.join(folder_root, 'input', 'vektor', 'batas_desa_citarum_48S.shp')
print(desa_polygon_file)
gdf = gpd.read_file(desa_polygon_file)

crs = gdf.crs
gdf['x_utm'] = gdf.centroid.x
gdf['y_utm'] = gdf.centroid.y

e:\citarum\GeoAnalisisGEE\input\vektor\batas_desa_citarum_48S.shp


In [39]:
list_raster_file = glob.glob(os.path.join(folder_root, 'input', 'raster', '*.tif'))
df_raster_file = pd.DataFrame(list_raster_file)
df_raster_file = df_raster_file.astype('string')
print(df_raster_file)
print(len(df_raster_file))
list(df_raster_file)
df_raster_file.rename(columns = {0:'filenamelong'}, inplace = True)

df_raster_file['filename'] = df_raster_file["filenamelong"].str.split("\\").str[-1]
df_raster_file['parameter'] = df_raster_file["filename"].str.split(".").str[0]
print(df_raster_file)


                                                   0
0  e:\citarum\GeoAnalisisGEE\input\raster\2020_de...
1  e:\citarum\GeoAnalisisGEE\input\raster\2020_de...
2  e:\citarum\GeoAnalisisGEE\input\raster\2020_et...
3  e:\citarum\GeoAnalisisGEE\input\raster\2020_fr...
4  e:\citarum\GeoAnalisisGEE\input\raster\2020_pa...
5  e:\citarum\GeoAnalisisGEE\input\raster\2020_pr...
6
                                        filenamelong                filename  \
0  e:\citarum\GeoAnalisisGEE\input\raster\2020_de...            2020_dem.tif   
1  e:\citarum\GeoAnalisisGEE\input\raster\2020_de...  2020_depth_to_root.tif   
2  e:\citarum\GeoAnalisisGEE\input\raster\2020_et...            2020_eto.tif   
3  e:\citarum\GeoAnalisisGEE\input\raster\2020_fr...         2020_fractp.tif   
4  e:\citarum\GeoAnalisisGEE\input\raster\2020_pa...           2020_pawc.tif   
5  e:\citarum\GeoAnalisisGEE\input\raster\2020_pr...  2020_precipitation.tif   

            parameter  
0            2020_dem  
1  2020_depth_to_r

In [42]:
vars = []
zstats_merged = []

for ind in df_raster_file.index:
    raster_data = rio.open(df_raster_file['filenamelong'][ind], "r")  
    parameter = df_raster_file["parameter"][ind]
    print(parameter)
    profile = raster_data.profile
    transform = profile['transform']
    nodata = raster_data.nodata

    img = raster_data.read(1)
    img = img.astype('float32') 
    img[img==nodata] = np.nan
    zstats = zonal_stats(gdf, img, affine=transform, prefix= f'{parameter}_', nodata=nodata, stats='mean')
    zstats_merged.append(zstats) # zstats_merged is now a 2D list of lists [bands, shapes]
    vars.append(f'{parameter}_mean')
 